In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3

In [2]:


hard = pd.read_csv("initial csv/hard23.csv")
clay = pd.read_csv("initial csv/clay23.csv")
grass = pd.read_csv("initial csv/grass23.csv")
overall23 = pd.read_csv("initial csv/2023overall.csv")

#Renaming the columns
# hard = [df.rename(columns={"value": "Hard_Win%"}) for df in hard]
# clay = [df.rename(columns={"value": "Clay_Win%"}) for df in clay]
# grass = [df.rename(columns={"value": "Grass_Win%"}) for df in grass]

hard = hard.rename(columns = {"value": "hard_Win%"})
clay = clay.rename(columns = {"value": "clay_Win%"})
grass = grass.rename(columns = {"value": "grass_Win%"})

overall23 = overall23.rename(columns = {"value": "Overall_Win%"})


# get tennis abstract dataset and see what the intersecting dataframes look like for hard, clay, and grass
clayserve = pd.read_csv("initial csv/clayserve.csv")
hardserve = pd.read_csv("initial csv/hardserve.csv")
grassserve = pd.read_csv("initial csv/grassserve.csv")
clayreturn = pd.read_csv("initial csv/clayreturn.csv")
hardreturn = pd.read_csv("initial csv/hardreturn.csv")
grassreturn = pd.read_csv("initial csv/grassreturn.csv")
rally = pd.read_csv("initial csv/rally.csv", encoding='windows-1252')

#edit player names to get rid of the country at end 
hardserve['Player'] = hardserve['Player'].str.slice(0,-6)
clayserve['Player'] = clayserve['Player'].str.slice(0,-6)
grassserve['Player'] = grassserve['Player'].str.slice(0,-6)

hardreturn['Player'] = hardreturn['Player'].str.slice(0,-6)
clayreturn['Player'] = clayreturn['Player'].str.slice(0,-6)
grassreturn['Player'] = grassreturn['Player'].str.replace('\xa0',' ', regex = False)
grassreturn['Player'] = grassreturn['Player'].str.slice(0,-6)
rally['Player'] = rally['Player'].str.replace('\xa0',' ', regex = False)

In [3]:
#rename fritz alcaraz rune 
def rename(df):
    df['name'] = df['name'].replace('Carlos Alcaraz Garfia', 'Carlos Alcaraz')
    df['name'] = df['name'].replace('Taylor Harry Fritz', 'Taylor Fritz')
    df['name'] = df['name'].replace('Holger Vitus Nodskov Rune', 'Holger Rune')
rename(hard)
rename(clay)
rename(grass)

In [4]:
#Now, let's merge all the 21 dfs together?
conn = sqlite3.connect("tennis.db")
hard.to_sql("hard", conn, if_exists = 'append', index = False)
clay.to_sql("clay", conn, if_exists = 'append', index = False)
grass.to_sql("grass", conn, if_exists = 'append', index = False)
hardserve.to_sql("hardserve", conn, if_exists = 'append', index = False)
clayserve.to_sql("clayserve", conn, if_exists = 'append', index = False)
grassserve.to_sql("grassserve", conn, if_exists = 'append', index = False)
hardreturn.to_sql("hardreturn", conn, if_exists = 'append', index = False)
clayreturn.to_sql("clayreturn", conn, if_exists = 'append', index = False)
grassreturn.to_sql("grassreturn", conn, if_exists = 'append', index = False)
rally.to_sql("rally", conn, if_exists = 'append', index = False)
conn.close()


In [5]:
# conn = sqlite3.connect("tennis.db")
# cursor = conn.cursor()

# #sql query
# cmd = \
# """ 
# SELECT H.name, H.country_name, H.'Hard_Win%', C.'Clay_Win%', G.'Grass_Win%'
# FROM hard H
# INNER JOIN clay C ON H.name = C.name
# INNER JOIN grass G on H.name = G.name;
# """
# tentative = pd.read_sql_query(cmd, conn) #read the query to a dataframe
# conn.close()
# tentative

# Functions for serve, return, and rally

In [6]:
def serve(surface):
    '''
    sql query that returns serve dataframe for specified surface
    '''
    conn = sqlite3.connect("tennis.db")

    #sql query
    cmd = \
    f""" 
    SELECT H.name, H.country_name, H.'{surface}_Win%', HS.'Ace%', HS.SPW , HS.'DF%', HS.'1st%', HS.'2nd%', HS.'Hld%', HS.'Pts/SG' 
    FROM {surface} H
    INNER JOIN {surface}serve HS ON H.name = HS.player
    """
    serve = pd.read_sql_query(cmd, conn)
    conn.close()
    return serve

In [7]:
def returns(surface):
    '''sql query that returns return dataframe for specified surface'''
    conn = sqlite3.connect("tennis.db")

    #sql query
    cmd = \
    f""" 
    SELECT H.name, H.country_name, H.'{surface}_Win%', HS.RPW, HS.'vAce%' , HS.'v1st%', HS.'v2nd%', HS.'Brk%', HS.'PtsW/RG'
    FROM {surface} H
    INNER JOIN {surface}return HS ON H.name = HS.player
    """
    returns = pd.read_sql_query(cmd, conn)
    conn.close()
    return returns

In [8]:
def rally(surface):
    conn = sqlite3.connect("tennis.db")
    #sql query
    cmd = \
    f""" 
    SELECT H.name, H.country_name, H.'{surface}_Win%', HS.RallyLen, HS.'RLen-Serve' , HS.'RLen-Return',
    HS.'1-3\xa0W%', HS.'4-6\xa0W%', HS.'7-9\xa0W%', HS.'10+\xa0W%', HS.'FH/GS', HS.'BH\xa0Slice%', HS.'FHP/Match', HS.'BHP/Match'
    FROM {surface} H
    INNER JOIN rally HS ON H.name = HS.Player
    """
    rally = pd.read_sql_query(cmd, conn)
    conn.close()
    return rally

In [9]:
hardserve = serve("hard")
clayserve = serve("clay")
grassserve = serve("grass")
hardreturns = returns("hard")
clayreturns = returns("clay")
grassreturns = returns("grass")
hardrally = rally("hard")
clayrally = rally("clay")
grassrally = rally("grass")

In [10]:
hardserve.to_csv("HardServeBSA.csv")
clayserve.to_csv("ClayServeBSA.csv")
grassserve.to_csv("GrassServeBSA.csv")

hardreturns.to_csv("HardReturnsBSA.csv")
clayreturns.to_csv("ClayReturnsBSA.csv")
grassreturns.to_csv("GrassReturnsBSA.csv")

hardrally.to_csv("HardRallyBSA.csv")
clayrally.to_csv("ClayRallyBSA.csv")
grassrally.to_csv("GrassRallyBSA.csv")